In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras.models import Sequential

In [2]:
TRAIN_DIR = '../data/processed/train/'
TEST_DIR = '../data/processed/test/'
VAL_DIR = '../data/processed/val/'
IMG_SIZE = (224,224)

In [3]:
# Load the dataset without any predefined batch size
train_data = tf.keras.utils.image_dataset_from_directory(
    directory = TRAIN_DIR,
    labels='inferred',
    label_mode='int',
    batch_size=None,  # Load as unbatched dataset
    image_size=IMG_SIZE,  # Resize images to standard size
    seed=42 
)

Found 125 files belonging to 2 classes.


In [4]:
test_data = keras.utils.image_dataset_from_directory(
    directory = TEST_DIR,
    labels = 'inferred',
    label_mode = 'int',
    color_mode = 'grayscale',
    batch_size = 120,
    image_size=IMG_SIZE,  # Resize images to standard size
    seed=42 
)

Found 27 files belonging to 2 classes.


In [5]:
val_data = keras.utils.image_dataset_from_directory(
    directory = VAL_DIR,
    labels = 'inferred',
    label_mode = 'int',
    color_mode = 'grayscale',
    batch_size = 120,
    image_size=IMG_SIZE,  # Resize images to standard size
    seed=42 
)

Found 27 files belonging to 2 classes.


In [6]:
train_data

<ShuffleDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [7]:
# Class names
class_names = ['tumor','no_tumor']

In [8]:
# Extract data from train dataset
def extract_images_and_labels(dataset):
    X, y = [], []
    for image, label in dataset:
        X.append(image.numpy())
        y.append(label.numpy())
    return np.array(X), np.array(y)

# Assuming train_data is loaded correctly
X_train, y_train = extract_images_and_labels(train_data)

2025-03-20 10:20:51.201589: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [9]:
X_val, y_val = extract_images_and_labels(val_data)

In [10]:
print(f'X shape: {X_train.shape}')
print(f'y shape: {y_train.shape}')

X shape: (125, 224, 224, 3)
y shape: (125,)


In [11]:
model_file = "my_model.keras"

model = Sequential()
model.add(tf.keras.layers.Rescaling(1./255, input_shape=(224,224,3)))
model.add(Conv2D(32, (3,3), padding="Same", activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (5,5), padding="Same", activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3,3), padding="Same", activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (5,5), padding="Same", activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3,3), padding="Same", activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (5,5), padding="Same", activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [13]:
model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 224, 224, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 32)      25632     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 112, 112, 32)      0

In [15]:
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))

Epoch 1/50
4/4 [==============================] - ETA: 0s - loss: 18.3332 - accuracy: 0.5120

ValueError: in user code:

    File "/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1820, in test_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1804, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1792, in run_step  **
        outputs = model.test_step(data)
    File "/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1756, in test_step
        y_pred = self(x, training=False)
    File "/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/input_s